# Ecuación de Navier Stokes (ENS)

Nota: a este documento lo podremos ejecutar de manera dinámica si tienen instalado:

- Python 3.5 o más nuevo instalado.
- [Jupyter Notebook](https://jupyter.readthedocs.io/en/latest/install.html).
- [FEniCS](https://fenicsproject.org/).

La visualización del mismo es óptima utilizando Jupyter Notebook.

### Referencias
- Larson y Bengzon, The Finite Element Method: Theory, Implementation, and Practice.
- Capítulo 20 y 21 de Logg , Mardal y Wells, Automated Solution of Differential Equations by the Finite Element Method.
- Utilizamos el ejemplo del tutorial de FEniCS, [seguir enlace](https://fenicsproject.org/pub/tutorial/html/._ftut1009.html).

### Introducción

En esta clase continuaremos con las ideas del [tutorial anterior](https://github.com/rirastorza/Intro2FEM/blob/master/Problemas_dependientes_del_tiempo/navier_stokes_parte1.ipynb). Desarrollaremos otro ejemplo en el cual se pueden visualizar vórtices y además se modifica la condición de borde del sistema.

### Formulación variacional de Ecuaciones de Navier-Stokes para un fluido newtoniano

Las ecuaciones son: 

$$ \rho\frac{\partial\mathbf{u}}{\partial t}+ \rho \mathbf{u}\cdot \nabla u  =  \nabla \cdot\sigma(u,p)+f  \space\space\space en \space\space\space \Omega \tag{1} $$

$$\nabla  \cdot \mathbf{u}  =  0 \space  \space\space\space\space\space\space en \space\space\space \Omega \tag{2}$$

$$\mathbf{u}  = \mathbf{u_{0}}  \space\space\space\space\space\space en \space\space\space \partial\Omega \tag{3}$$

Recuerden $\Omega \subset R^{d} $ y $\partial\Omega \subset R^{d-1}$. Ejemplo, si d= 2 implicaría que estaríamos resolviendo las ecuaciones para un sistema $\left(x,y\right)$ y las condicines de borde se las estaríamos aplicando a una línea de ese sistema.

Notar que $ \mathbf{u}  = \mathbf{u_{0} = 0}$ indica que la velocidad contra la pared del tubo o canal vale cero, esta es una condición muy usada. Aunque si el fluido no moja la pared (imaginar superfice de teflon) esa velocidad no es nula, hay gran discucion en esos casos de cual sería el valor de la velocidad, a esa condición de deslizamiento se le llama "slipash" en la literatura.

Recordemos también que: 

$$\sigma(u,p)= 2 \mu \epsilon(u) - p I$$

donde $\sigma(u,p)$ es el tensor de tensiones,$\mu$ es la viscosidad, y $\epsilon(u)$ tensor de velocidad de deformación definido como siempre:

$$\epsilon(u) = \frac{1}{2}(\nabla u + (\nabla u)^T).$$

Aplicaremos nuevamente el método conocido como **método de división**, en particular, una variante del método propuesto por *Chorin et. al. (Numerical solution of the Navier-Stokes equations, Math. Comp., 22, pp. 745-762, 1968)*: esquema de **corrección de presión inxcremental (Incremental Pressure Correction Scheme IPCS)**. Consta de tres pasos que repasaremos a continuación: 

#### Paso (1)

Calculamos una velocidad tentativa $ \mathbf{u}^{*}$ avanzando la ecuación de cantidad de movimiento, Ec. (1), $\rho\frac{\partial\mathbf{u}}{\partial t}+ \rho \mathbf{u}\cdot \nabla \mathbf{u}  =  \nabla \cdot\sigma\left(\mathbf{u},p\right)+f $ 

mediante un esquema de diferencia finita de punto medio en el tiempo, pero usando la presión $p^{n}$ del intervalo de tiempo anterior, es decir, el tiempo $t^{n}$. También linealizaremos el término convectivo no lineal usando la velocidad conocida $\mathbf{u}^{n}$ del paso de tiempo anterior, $u^{n}\cdot\nabla u^{n}$. El problema variacional para este primer paso es:

$$  \langle\frac{\rho(\mathbf{u}^{*}-\mathbf{u}^{n}}{\Delta t}),v\rangle +  \langle\rho(\mathbf{u}^{n}\cdot \nabla \mathbf{u}^{n},v\rangle + \langle\sigma(\mathbf{u}^{n+1/2}, p^{n}), \epsilon(v)\rangle+ \langle p^{n}n,v\rangle_{\partial\Omega} - \langle\mu \Delta \mathbf{u}^{n+1/2}\cdot n ,v\rangle_{\partial\Omega}= \langle  f^{n+1}, v\rangle \tag{paso 1}$$
 
Notar que $\mathbf{u}^{n+1/2} \approx (\mathbf{u}^{n} + \mathbf{u}^{n+1})/2$, además debe incluir las condiciones de borde para la velocidad.
 
 #### Paso (2)
 Con la ayuda de $\mathbf{u}^{*}$ calculamos $p^{n+1}$ de la siguiente manera:
 
 $$  \langle \nabla p^{n+1}, \nabla q\rangle =   \langle \nabla p^{n}, \nabla q\rangle - \langle \left(\nabla \cdot \mathbf{u}^{*}, q\right)/ \Delta t \rangle \tag{paso 2}$$
 
recordemos que $q$ era la función de prueba para la presión. Acá también es importante y se deben incluir las condiciones de borde para la presión.
 
 #### Paso (3) 
Reemplazamos en la ecuación de conservación de momento, Ec. (1), $\mathbf{u}^{*}, p^{n}$  y la restamos a la resultante de reemplazar en la misma ecuación $\mathbf{u}^{n+1}, p^{n+1}$, el resultado es:
 
 $$\frac{\mathbf{u}^{n+1}- \mathbf{u}^{*}}{\Delta t} + \nabla p^{n+1} - \nabla p^{n} =0 \tag{paso 3}$$
 
De la ecuación del Paso (3) ¿qué conocemos y que desconocemos? Repasemos:

$ \mathbf{u}^{*} $ velocidad estimada en el Paso (1) $ p^{n} $ es conocida y $ p^{n+1} $ calculada en el Paso (2).
 
Ahora viene la parte dónde le exijo a la solución satisfacer la Ecuación de conservación de la masa, o sea la ecuación de continuidad, Ec. (2), $\nabla  \cdot \mathbf{u}^{n+1}  =  0 $, entonces:
 
 $$\frac{-\nabla \cdot \mathbf{u}^{*}}{\Delta t} + \nabla^{2} p^{n+1} - \nabla^{2} p^{n} =0 \tag{paso 3}$$
 
Recordems que $\mathbf{u}^{*}$ es conocida y que la presión es un escalar, entonces el paso (3) resulta en el problema de Poisson para la $p^{n+1}$, finalmente es posible entonces calcular $\mathbf{u}^{n+1}$ de la siguiente ecuación:
 
$$  \langle \mathbf{u}^{n+1}, v\rangle =  \langle \mathbf{u}^{*}, v\rangle - \Delta t \langle \nabla (p^{n+1}-p^{n}), v\rangle $$
 
Resumen: el método logra resolver eficientemente las Ecuaciones de Navier-Stokes para un fluido incompresible resolviendo tres problemas variacionales lineales en cada paso de tiempo, uno para la velocidad, otro para la presión y la ecuación de continuidad que hace que se cumplan efectivamente las dos ecuaciones de balance: **la conservación de la cantidad de movimiento y la conservación de la masa**. 

### Ejemplo

Hemos modificado el ejemplo de FEniCS *ft08_navier_stokes_cylinder.py* actualizando algunas sentencias y modificando otras para que nos funcione. Todo está subido en la carpeta ejemplos con el nombre *ejemplo21.py*.

Las diferencias más relevantes respecto del ejemplo anterior son: **la geometría y la condición de contorno**, comenzaremos por la geometría. Se utiliza una geometría (sacada de este [enlace](http://www.featflow.de/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html)) que permite estudiar la performance del FEM en dinámica de un fluido Newtoniano.

[<img src="geometriaNS2.png" width="600"/>](geometriaNS2.png)

En el gráfico anterior se muestra también la condición de borde de la velocidad en la izquierda, cuya expresión es:

$$ \mathbf{u}\left(0,y\right)=\frac{4\times 1.5 y \left(0.41-y \right)}{0.41^{2}} $$

esto es un perfil parabólico de entrada.

La figura también muestra la malla, que la construimos utilizando *mshr* restando una circunferencia de un rectángulo.

In [ ]:
from __future__ import print_function
from fenics import *
from mshr import *
import numpy as np

T = 5.0            # tiempo final
num_steps = 5000   # número de pasos
dt = T / num_steps # tamaño de paso de tiempo
mu = 0.001         # viscosidad
rho = 1            # densidad

# Creo la malla
channel = Rectangle(Point(0, 0), Point(2.2, 0.41))
cylinder = Circle(Point(0.2, 0.2), 0.05)
domain = channel - cylinder
mesh = generate_mesh(domain, 64)

La mayoría del script es similar al *ejemplo20.py*, aquí no repetiremos todo. Noten las condiciones de borde como se pueden definir:

In [ ]:
# Condiciones de borde
inflow   = 'near(x[0], 0)'
outflow  = 'near(x[0], 2.2)'
walls    = 'near(x[1], 0) || near(x[1], 0.41)'
cylinder = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'

La referida al cilindro, donde debemos imponer $ \mathbf{u}  = \mathbf{u_{0} = 0}$ como en las paredes inferior y superior: condición de slipash, se escribe de una manera diferente. Lo que significa esa expresión es lo siguiente: si FEniCS encuentra un borde en esa región, entonces tendrá la condición que le impongamos a *cylinder*. En este caso, el borde en esa región ($0.1<x<0.3$ e $0.1<y<0.3$) es la circunferencia.

Además el perfil de velocidad de entrada, es un vector que solo tiene componente $x$.

In [ ]:
inflow_profile = ('4.0*1.5*x[1]*(0.41 - x[1]) / pow(0.41, 2)', '0')

Ahora veamos como se introduce esto:

In [ ]:
bcu_inflow = DirichletBC(V, Expression(inflow_profile, degree=2), inflow)
bcu_walls = DirichletBC(V, Constant((0, 0)), walls)
bcu_cylinder = DirichletBC(V, Constant((0, 0)), cylinder)
bcp_outflow = DirichletBC(Q, Constant(0), outflow)
bcu = [bcu_inflow, bcu_walls, bcu_cylinder]
bcp = [bcp_outflow]

El resto es lo mismo que el otro ejemplo, con la diferencia que aquí guardaremos los datos para poder graficarlos. A diferencia del tutorial de FEniCS aquí lo guardaremos en un archivo pvd.

In [ ]:
pdvfile_p = File("data/presion.pvd")
pdvfile_u = File("data/velocidad.pvd")

Otra **diferencia importante** respecto del *ejemplo20.py* es el solver que vamos a usar. Esto es así porque los sistemas lineales que vamos a utilizar son considerablemente más grandes. El esquema que se utilizará es uno iterativo, es decir, cada paso resolverá el sistema lineal con un método iterativo que es mejor para matrices ralas. Para poder hacer esto se necesitan dos cosas:

- Un precondicionador
- Un método iterativo del [subespacio de Krylov](https://en.wikipedia.org/wiki/Krylov_subspace)

Aquí se utilizarán los del tutorial: el método del Gradiente Biconjugado Estabilizado [BiCGSTAB](https://es.wikipedia.org/wiki/M%C3%A9todo_del_gradiente_biconjugado_estabilizado) y el Método del Gradiente Conjugado. En el caso del BiCGSTAB el precondicionador es *'hypre_amg'* ([Hypre algebraic multigrid (BoomerAMG)](https://hypre.readthedocs.io/en/latest/solvers-boomeramg.html)) y en el caso del Gradiente Conjugado es el *'sor'* (successive over-relaxation). Alguna información más [aquí](https://fenicsproject.org/pub/tutorial/html/._ftut1017.html#ftut:app:solver:prec).


In [ ]:
   # Paso 1: Tentative velocity step
    b1 = assemble(L1)
    [bc.apply(b1) for bc in bcu]
    solve(A1, u_.vector(), b1, 'bicgstab', 'hypre_amg')

    # Paso 2: Pressure correction step
    b2 = assemble(L2)
    [bc.apply(b2) for bc in bcp]
    solve(A2, p_.vector(), b2, 'bicgstab', 'hypre_amg')

    # Paso 3: Velocity correction step
    b3 = assemble(L3)
    solve(A3, u_.vector(), b3, 'cg', 'sor')

Para ver los solvers y los precondicionadores que existen en FEniCS se pueden escribir las siguientes sentencias:

In [ ]:
list_linear_solver_methods()
list_krylov_solver_preconditioners()

Las figuras que debería obtener si ejecuta el script son las siguientes:

[<img src="velocidadNS2.png" width="950"/>](velocidadNS2.png)
[<img src="presionNS2.png" width="1000"/>](presionNS2.png)

### Comentarios

En el capítulo 21 de Logg , Mardal y Wells, Automated Solution of Differential Equations by the Finite Element Method se muestran otros esquemas para resolver la dinámica de fluidos incompresibles. Todos los ejemplos están implementados en un conjunto de scripts denominados *NSbench*. Los esquemas disponibles son los siguientes:

- Chorin
- IPCS
- Consistent splitting scheme (css1)
- Consistent splitting scheme (css2)
- A least-squares stabilized Galerkin method (g2)
- Generalized Richardson iteration on the pressure Schur complement (grpc)

de los cuales hemos visto el IPCS. Además, tiene varios problemas clásicos para resolver.

Si bien los scripts están desactualizados y no los mantienen, es interesante estudiarlos porque esencialmente no ha cambiado mucho, y gran parte del código se puede reutilizar.

En el siguiente [enlace](https://bazaar.launchpad.net/~nsbench/nsbench/main/files) se encuentra el código.
